In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [5]
commitnumber = de5d43e
molecule = {'co2': 0.0004}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band3c
  atmpro = mls
  band = 3c
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.375563e-24
  molecule = co2
  ng_adju = [0, 0, 0]
  ng_refs = [3, 2, 4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 800
  vmin = 720
  w_diffuse = [(1.7, 1.6, 1.8), (1.8, 1.7), (1.5, 1.6, 1.7, 1.8)]
  wgt = [(0.6, 0.4, 0.7), (0.7, 0.4), (0.3, 0.4, 0.85, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.058118                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.040641
0.000750      2.0  0.078796                  2.0 -0.021163
0.001052      3.0  0.141390                  3.0  0.025413
0.001476      4.0  0.211531                  4.0  0.079058
0.002070      5.0  0.287482                  5.0  0.139387
0.002904      6.0  0.368151                  6.0  0.206417
0.004074      7.0  0.451561                  7.0  0.281653
0.005714      8.0  0.534892                  8.0  0.364641
0.008015      9.0  0.614633                  9.0  0.453088
0.011243     10.0  0.686432                 10.0  0.544692
0.015771     11.0  0.745496                 11.0  0.636058
0.022122     12.0  0.787649                 12.0  0.722340
0.031031     13.0  0.810169                 13.0  0.796031
0.043528     14.0  0.792132                 14.0  0.829870
0.061057     15.0  0.720423                 15.0  0.798484
0.085645     16.0  0.627359                 16.0  0.716303
0.120136     17.0  0.541204                 17.0  0.606216
0.168516     18.0  0.469413                 18.0  0.487306
0.236378     19.0  0.417222                 19.0  0.387828
0.331549     20.0  0.391370                 20.0  0.336877
0.465100     21.0  0.407800                 21.0  0.358391
0.652400     22.0  0.364736                 22.0  0.340043
0.915100     23.0  0.256676                 23.0  0.254815
1.283650     24.0  0.184515                 24.0  0.193434
1.800600     25.0  0.143089                 25.0  0.152454
2.525700     26.0  0.112057                 26.0  0.118713
3.542800     27.0  0.084788                 27.0  0.091615
4.969550     28.0  0.065987                 28.0  0.074774
6.970850     29.0  0.057454                 29.0  0.068791
9.778100     30.0  0.050825                 30.0  0.062072
13.715850    31.0  0.042472                 31.0  0.050234
19.239350    32.0  0.032976                 32.0  0.034950
26.987250    33.0  0.028533                 33.0  0.025372
37.855300    34.0  0.029459                 34.0  0.023413
53.100050    35.0  0.029553                 35.0  0.024605
73.887500    36.0  0.029254                 36.0  0.028536
97.662500    37.0  0.028859                 37.0  0.031232
121.437500   38.0  0.026439                 38.0  0.029823
145.212500   39.0  0.022978                 39.0  0.026085
168.987500   40.0  0.020016                 40.0  0.022254
192.762500   41.0  0.017013                 41.0  0.018179
216.537500   42.0  0.013773                 42.0  0.013845
240.312500   43.0  0.010077                 43.0  0.008977
264.087500   44.0  0.005370                 44.0  0.003101
287.862500   45.0 -0.001114                 45.0 -0.004303
311.637500   46.0 -0.003347                 46.0 -0.007124
335.412500   47.0  0.002515                 47.0 -0.001474
359.187500   48.0  0.010272                 48.0  0.006408
382.962500   49.0  0.016711                 49.0  0.013136
406.737500   50.0  0.022659                 50.0  0.019277
430.512500   51.0  0.028386                 51.0  0.025124
454.287500   52.0  0.033977                 52.0  0.030823
478.062500   53.0  0.039465                 53.0  0.036487
501.837500   54.0  0.044845                 54.0  0.042100
525.612500   55.0  0.050089                 55.0  0.047583
549.387500   56.0  0.055132                 56.0  0.053052
573.162500   57.0  0.059885                 57.0  0.058398
596.937500   58.0  0.064242                 58.0  0.063475
620.712500   59.0  0.068093                 59.0  0.068115
644.487500   60.0  0.071361                 60.0  0.072189
668.262500   61.0  0.074022                 61.0  0.075636
692.037500   62.0  0.076144                 62.0  0.078604
715.812500   63.0  0.077848                 63.0  0.081118
739.587500   64.0  0.079315                 64.0  0.083345
76

# Fluxes by Layer

CRD                          CLIRAD            \
                         flug      fldg      fnetg       flug      fldg   
pressure     level                                                        
0.000000e+00 1     -16.429054  0.000000 -16.429054        NaN       NaN   
1.000000e-08 1            NaN       NaN        NaN -16.325951  0.000001   
6.244000e-04 2     -16.429086  0.000036 -16.429050 -16.325979  0.000026   
8.759000e-04 3     -16.429098  0.000051 -16.429047 -16.325991  0.000037   
1.228600e-03 4     -16.429116  0.000074 -16.429042 -16.326006  0.000054   
1.723400e-03 5     -16.429139  0.000110 -16.429029 -16.326028  0.000080   
2.417400e-03 6     -16.429171  0.000166 -16.429006 -16.326057  0.000121   
3.390900e-03 7     -16.429214  0.000251 -16.428963 -16.326096  0.000184   
4.756500e-03 8     -16.429271  0.000381 -16.428890 -16.326148  0.000281   
6.672000e-03 9     -16.429345  0.000576 -16.428769 -16.326216  0.000432   
9.358900e-03 10    -16.429439  0.000866 -16.428573 -16.326305  0.000665   
1.312780e-02 11    -16.429557  0.001291 -16.428267 -16.326416  0.001019   
1.841450e-02 12    -16.429701  0.001901 -16.427800 -16.326551  0.001553   
2.583020e-02 13    -16.429868  0.002760 -16.427108 -16.326709  0.002345   
3.623230e-02 14    -16.430047  0.003937 -16.426110 -16.326880  0.003497   
5.082340e-02 15    -16.430225  0.005485 -16.424740 -16.327052  0.005103   
7.129060e-02 16    -16.430403  0.007409 -16.422994 -16.327224  0.007212   
1.000000e-01 17    -16.430577  0.009716 -16.420860 -16.327389  0.009814   
1.402710e-01 18    -16.430715  0.012437 -16.418278 -16.327518  0.012834   
1.967600e-01 19    -16.430761  0.015624 -16.415137 -16.327553  0.016130   
2.759970e-01 20    -16.430599  0.019377 -16.411221 -16.327384  0.019602   
3.871000e-01 21    -16.429959  0.023888 -16.406070 -16.326747  0.023399   
5.431000e-01 22    -16.427987  0.029453 -16.398535 -16.324759  0.028035   
7.617000e-01 23    -16.424388  0.035298 -16.389090 -16.321038  0.033120   
1.068500e+00 24    -16.420204  0.040442 -16.379762 -16.316627  0.037970   
1.498800e+00 25    -16.415927  0.045570 -16.370356 -16.312035  0.043240   
2.102400e+00 26    -16.411575  0.051449 -16.360125 -16.307279  0.049385   
2.949000e+00 27    -16.407252  0.058364 -16.348888 -16.302464  0.056477   
4.136600e+00 28    -16.403412  0.066452 -16.336960 -16.297997  0.064899   
5.802500e+00 29    -16.400425  0.076487 -16.323938 -16.294182  0.075841   
8.139200e+00 30    -16.398018  0.089983 -16.308035 -16.290671  0.091373   
1.141700e+01 31    -16.396184  0.107883 -16.288301 -16.287454  0.112261   
1.601470e+01 32    -16.395861  0.130692 -16.265169 -16.285592  0.137761   
2.246400e+01 33    -16.399567  0.159591 -16.239976 -16.287945  0.166817   
3.151050e+01 34    -16.409110  0.199710 -16.209400 -16.296507  0.202571   
4.420010e+01 35    -16.424041  0.258923 -16.165118 -16.310278  0.251539   
6.200000e+01 36    -16.445711  0.342906 -16.102804 -16.330697  0.323843   
8.577500e+01 37    -16.475025  0.454611 -16.020414 -16.359068  0.432589   
1.095500e+02 38    -16.504632  0.565492 -15.939139 -16.388506  0.549994   
1.333250e+02 39    -16.536534  0.671856 -15.864678 -16.420993  0.666481   
1.571000e+02 40    -16.573214  0.773248 -15.799966 -16.458879  0.777839   
1.808750e+02 41    -16.615181  0.871587 -15.743594 -16.502716  0.884356   
2.046500e+02 42    -16.663035  0.967354 -15.695680 -16.553309  0.986152   
2.284250e+02 43    -16.717653  1.060761 -15.656892 -16.612253  1.084091   
2.522000e+02 44    -16.780443  1.151933 -15.628511 -16.681293  1.178415   
2.759750e+02 45    -16.854411  1.241023 -15.613388 -16.763537  1.269393   
2.997500e+02 46    -16.946469  1.329946 -15.616524 -16.865317  1.359052   
3.235250e+02 47    -17.057272  1.431323 -15.625950 -16.986147  1.459816   
3.473000e+02 48    -17.177751  1.558884 -15.618867 -17.116112  1.585628   
3.710750e+02 49    -17.301950  1.712011 -15.589939 -17.249236  1.736802   
3.948500e+02 50    -17.428519  1.885643 -15.542876 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')